<a href="https://colab.research.google.com/github/ShirleyW-W/PSTAT134-Project/blob/charles/BERTFakeNewsClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using a Pre-trained BERT Model for Text Classification

## Loading Packages

In [1]:
#!pip install numpy==1.23.5
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np
import pandas as pd
import os
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## Import Real and Fake News Dataset

In [2]:
# !pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset")

print("Path to dataset files:", path)

100%|██████████| 41.0M/41.0M [00:02<00:00, 14.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/clmentbisaillon/fake-and-real-news-dataset/versions/1


In [3]:
class BERTZeroShotClassifier:
    def __init__(self, candidate_labels=None):
        self.model_name = "facebook/bart-large-mnli"
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name)

        self.model.eval()

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # Define the candidate labels for zero-shot classification
        if candidate_labels is None:
            self.candidate_labels = [
                {"real": ["factual news", "verified information"]},
                {"fake": ["fake news", "misinformation"]}
            ]
        else:
            self.candidate_labels = candidate_labels

        self.all_labels = []
        for category_dict in self.candidate_labels:
            for category, label_list in category_dict.items():
                for label in label_list:
                    self.all_labels.append(label)

    def classify_text(self, article_text, threshold=0.5):
        """
        Classify text using zero-shot classification with multiple label categories

        Arguments:
            article_text (str): the text of the news article
            threshold (float): Confidence threshold for classification

        Returns:
            dict: classification results with categories, scores, and overall classification
        """

        # prepare inputs for the model
        inputs = self.tokenizer(
            f"This is a news article: {article_text}",
            return_tensors="pt",
            truncation=True,
            padding=True
        )

        #prepare label inputs
        label_inputs = []
        for label in self.all_labels:
            label_inputs.append(f"This news article contains {label}.")

        label_encodings = self.tokenizer(
            label_inputs,
            return_tensors="pt",
            truncation=True,
            max_length=128,
            padding=True
        )

        #move inputs to device
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        label_encodings = {k: v.to(self.device) for k, v in label_encodings.items()}

        #perform classification
        with torch.no_grad():
            text_embedding = self.model(**inputs).logits

            #get embeddings for each label
            label_embeddings = []
            for i in range(len(self.all_labels)):
                label_inputs = {
                    'input_ids': label_encodings['input_ids'][i].unsqueeze(0),
                    'attention_mask': label_encodings['attention_mask'][i].unsqueeze(0)
                }
                label_embedding = self.model(**label_inputs).logits
                label_embeddings.append(label_embedding)

            # calculate similarity scores
            scores = []
            for label_embedding in label_embeddings:
                similarity = torch.nn.functional.cosine_similarity(text_embedding, label_embedding)
                scores.append(similarity.item())


            #normalize scores
            scores = np.array(scores)
            scores = (scores - scores.min()) / (scores.max() - scores.min())

            results = {
                "label_scores": {label: float(score) for label, score in zip(self.all_labels, scores)},
                "categories": {}
            }

            label_index = 0
            for category_dict in self.candidate_labels:
                for category, label_list in category_dict.items():
                    category_scores = []
                    for _ in label_list:
                        category_scores.append(scores[label_index])
                        label_index += 1

                        results["categories"][category] = float(np.mean(category_scores))

            top_category = max(results["categories"].items(), key=lambda x: x[1])
            results["classification"] = top_category[0]
            results["confidence"] = top_category[1]
            results["is_confident"] = top_category[1] > threshold

            if not results["is_confident"]:
              results["classification"] = "unknown"

            return results

    def classify_batch(self, article_texts, threshold=0.5):
        """
        Classify multiple news articles

        Arguments:
            article_texts (list): list of article text strings
            threshold (float): confidence threshold

        Returns:
            list: list of classification results
        """
        return [self.classify_text(article, threshold) for article in article_texts]

In [4]:
def load_and_preprocess_dataset(fake_path, real_path, sample_size=None, seed=0):
    fake_df = pd.read_csv(fake_path)
    real_df = pd.read_csv(real_path)

    # add label column
    fake_df['label'] = 'fake'
    real_df['label'] = 'real'

    # Combine textual information of subject, title, and article text
    fake_df['content'] = "Subject: " + fake_df['subject'] + ", Title: " + fake_df['title'] + " " + fake_df['text']
    real_df['content'] = "Subject: " + real_df['subject'] + ", Title: " + real_df['title'] + " " + real_df['text']

    # select relevant columns: "article content", "label"
    fake_df = fake_df[['content', 'label']]
    real_df = real_df[['content', 'label']]

    #sample if needed
    if sample_size:
        fake_df = fake_df.sample(sample_size, random_state = seed)
        real_df = real_df.sample(sample_size, random_state = seed)

    # combine datasets
    combined_df = pd.concat([fake_df, real_df], ignore_index = True)

    # shuffle dataset
    combined_df = combined_df.sample(frac=1, random_state=seed).reset_index(drop=True)

    # apply parallel preprocessing for texts
    combined_df = parallel_preprocess_dataset(combined_df)

    return combined_df

In [9]:
def preprocess_text(text):
  """
  preprocess text by performing tokenization, stop word removal, lemmatization and stemming

  Arguments:
      text (str): input text to preprocess

  Returns:
      str: preprocessed text
  """

  from nltk.corpus import stopwords
  from nltk.stem import WordNetLemmatizer, PorterStemmer
  from nltk.tokenize import word_tokenize
  import re

  text = text.lower()

  text = re.sub(r'http\S+|www\S+|https\S+', '', text)

  # Remove special characters and numbers
  text = re.sub(r'[^a-zA-Z\s]', '', text)

  tokens = word_tokenize(text)

  stop_words = set(stopwords.words('english'))
  filtered_tokens = [word for word in tokens if word not in stop_words]

  # lemmatization
  lemmatizer = WordNetLemmatizer()
  lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

  # stemming
  stemmer = PorterStemmer()
  stemmed_tokens = [stemmer.stem(word) for word in lemmatized_tokens]

  preprocessed_text = ' '.join(stemmed_tokens)

  return preprocessed_text

In [6]:
def parallel_preprocess_dataset(df, n_jobs=-1):
  """
  Apply preprocessing to the content column of a dataframe in parallel

  Arguments:
    df (pd.DataFrame): DataFrame with 'content' column
    n_jobs (int): Number of jobs to run in parallel. -1 means using all processors

  Returns:
    pd.DataFrame: DataFrame with preprocessed 'processed_content' column
  """

  from joblib import Parallel, delayed
  import nltk

  try:
      nltk.data.find('tokenizers/punkt')
      nltk.data.find('tokenizers/punkt_tab')
      nltk.data.find('corpora/stopwords')
      nltk.data.find('corpora/wordnet')
  except LookupError:
      nltk.download('punkt')
      nltk.download('punkt_tab')
      nltk.download('stopwords')
      nltk.download('wordnet')

  processed_df = df.copy()

  processed_texts = Parallel(n_jobs=n_jobs)(
      delayed(preprocess_text)(text) for text in processed_df['content'].values
  )

  processed_df['processed_content'] = processed_texts

  return processed_df

In [7]:
def evaluate_model(classifier, test_df, threshold=0.5):
    """
    Evaluate the classifier on the test dataset

    Arguments:
        classifier (BertZeroShotClassifier): the classifier
        test_df (pd.DataFrame): test dataset

    Returns:
        dict: Evaluation metrics
    """

    predictions = []
    confidences = []

    results = classifier.classify_batch(test_df['content'].tolist(), threshold)
    predictions = [result['classification'] for result in results]
    confidences = [result['confidence'] for result in results]

    # calculate metrics
    accuracy = accuracy_score(test_df['label'], predictions)
    report = classification_report(test_df['label'], predictions, output_dict=True)
    conf_matrix = confusion_matrix(test_df['label'], predictions, labels = ['real', 'fake'])

    # add predictions and confidences to result dataframe
    results_df = test_df.copy()
    results_df['predicted'] = predictions
    results_df['confidence'] = confidences

    # identify misclassified articles
    results_df['misclassified'] = results_df['label'] != results_df['predicted']

    return {
        'accuracy': accuracy,
        'classification_report': report,
        'confusion_matrix': conf_matrix,
        'results_df': results_df
    }

## Execution

In [18]:
if __name__ == "__main__":
    fake_news_path = path + "/Fake.csv"
    real_news_path = path + "/True.csv"

    dataset = load_and_preprocess_dataset(fake_news_path, real_news_path, sample_size = 100, seed = 45)

    # create customized labels
    candidate_labels = [
        {"real": ["credible reporting", "fact-checked information", "objective journalism", "satire"]},
        {"fake": ["misinformation"]}
    ]
    # initialize the classifier
    classifier = BERTZeroShotClassifier(candidate_labels=candidate_labels)

    # evaluate the model
    print("Evaluating the model...")
    eval_results = evaluate_model(classifier, dataset, threshold=0.3)

    print("\nResults:")
    print(f"Accuracy: {eval_results['accuracy']:.4f}")
    print("\nClassification Report:")
    print(pd.DataFrame(eval_results['classification_report']).T)

    """
      classifying with new, more comprehensive candidate labels
    """

    dataset = load_and_preprocess_dataset(fake_news_path, real_news_path, sample_size = 100, seed = 45)

    # create customized labels
    expanded_candidate_labels = [
        {
            "real": [
                "factual or trustworthy news",
                "verified or authenticated information",
                "accurate or professional reporting",
                "credible, ethical, or objective journalism",
                "evidence-based or unbiased reporting",
                "fact-checked content",
                "reliable or moderate sources",
                "substantiated claims",
                "balanced coverage"
            ]
        },
        {
            "fake": [
                "fake news",
                "misinformation",
                "false reporting",
                "fabricated content",
                "misleading information",
                "unverified claims",
                "propaganda",
                "deceptive content",
                "clickbait",
                "hoax",
                "disinformation",
                "manipulated facts",
                "sensationalism",
                "conspiracy theory",
                "rumor"
            ]
        }
    ]

    # initialize the classifier
    classifier = BERTZeroShotClassifier(candidate_labels=expanded_candidate_labels)

    # evaluate the model
    print("Evaluating the model...")
    eval_results = evaluate_model(classifier, dataset, threshold=0.3)

    print("\nResults:")
    print(f"Accuracy: {eval_results['accuracy']:.4f}")
    print("\nClassification Report:")
    print(pd.DataFrame(eval_results['classification_report']).T)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Evaluating the model...

Results:
Accuracy: 0.4500

Classification Report:
              precision  recall  f1-score  support
fake           0.470588    0.80  0.592593   100.00
real           0.333333    0.10  0.153846   100.00
accuracy       0.450000    0.45  0.450000     0.45
macro avg      0.401961    0.45  0.373219   200.00
weighted avg   0.401961    0.45  0.373219   200.00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Evaluating the model...

Results:
Accuracy: 0.5550

Classification Report:
              precision  recall  f1-score  support
fake           0.554455   0.560  0.557214  100.000
real           0.555556   0.550  0.552764  100.000
accuracy       0.555000   0.555  0.555000    0.555
macro avg      0.555006   0.555  0.554989  200.000
weighted avg   0.555006   0.555  0.554989  200.000
